![lyon2 geonum](https://perso.liris.cnrs.fr/lmoncla/GEONUM/fig/logos.png)

# 2F2 – Gestion et traitement des données spatio-temporelles


## Tutoriel : Analyse des données des disponibilités des stations Vélo'v de la Métropole de Lyon


# Partie 1 : Exploration de données

Le cours a été réalisé par Ludovic Moncla - Il met à disposition sur sa plateforme des données du Grand Lyon qui ont été retravaillées.

L'objectif de ce tutoriel est d'appréhender la problématique d'analyse de données spatio-temporelles grâce à l'utilisation de librairies Python.
Pour cela nous allons travailler sur un cas d'étude visant la visualisation et le traitement des données de disponibilités des stations Vélo'v de la Métropole de Lyon. 

Les données que nous allons utilisées proviennent de la [plateforme data du Grand Lyon](https://data.grandlyon.com). 


Dans le cadre de ce TP,  vous avez à votre disposition l'ensemble des données pour la période du 7 octobre 2020 au 31 janvier 2021.


Les objectifs de ce tutoriel sont les suivants : 

* Récupérer le jeu de données, analyser sa structure et le charger dans un dataframe
* Explorer et visualiser les données
* Analyser les données : requêter le jeu de données pour générer des graphiques, des cartes et des cartes animées.






## 1. Configurer l'environnement



### 1.1 Importer les librairies

In [108]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import geopandas as gpd
import wget

## 2. Récupération du jeu de données

Pour palier la limite des 7 jours de disponibilité sur le site du Grand Lyon, Ludovic Moncla a développé un script qui récupère et stocke automatiquement les données chaque jour. Vous aurez ainsi accès aux données pour l'ensemble de l'année 2021. 
Les données sont au format CSV (plus simple à charger dans un dataframe qui le format JSON d'origine). Nous verrons la transformation du format de données lors de la dernière séance.

* Télécharger les archives contenant les données
1. data-stations.zip
2. data-bikes.zip

Ces 2 archives contiennent chacune un fichier CSV contenant respectivement la liste des stations vélov (et leur localisation) et la liste des disponibilités de chaque station par tranche de 30 minutes.


In [ ]:
## On télécharge l'archive contenant la liste des stations
wget.download("https://perso.liris.cnrs.fr/lmoncla/GEONUM/data-stations.zip",out="../data/")
    
## On télécharge l'archive contenant la liste des disponibilité des stations par tranche de 5 minutes
wget.download("https://perso.liris.cnrs.fr/lmoncla/GEONUM/data-bikes.zip",out="../data")

### 2.1. Chargement des données

Dans ce tutoriel nous n'allons pas utiliser de SGBD. L'objectif est de charger l'ensemble des données en mémoire dans une structure Python et de l'interroger directement. 

On distingue deux types de données :
1. les stations vélo'v (id station, latitude, longitude),
2. leurs historiques (id station, année, mois, jour, heure, minute, date, vélos disponibles, places disponibles).

Pour manipuler ces données nous allons utiliser les [dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) de la librairie Pandas.

Pandas est une librairie Python spécialisée dans l'analyse et la manipulation de données. Elle fourni en particulier un objet de type 'dataframe' qui permet de réaliser des opérations de prétraitement et de filtrage que nous utiliserons pour requêter les données.

Les premiers objectifs sont les suivants :

1. Stocker dans un premier dataframe la liste des stations velo'v et leurs coordonnées latitude / longitude associées.
2. Stocker dans un second dataframe pour chaque station et chaque pas de temps les données suivantes : 
    * id de la station
    * année
    * mois
    * jour
    * heure
    * minute
    * date complète (format d'origine)
    * nombre de vélos disponibles
    * nombre de places libres
    * nombre de départs des 30 dernières minutes
    * nombre d'arrivées dess 30 dernières minutes


Pour charger les données il suffit d'utiliser la méthode [read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv) de la librairie `Pandas`. Elle prend en paramètre le chemin du fichier que l'on souhaite charger. Ce fichier peut être de 2 formats, soit directement un fichier CSV, soit un fichier ZIP contenant un CSV. Dans notre, cas il est donc inutile de dézipper les archives téléchargées précédemment.


In [3]:
## On charge les données des stations dans un dataframe
df_stations = pd.read_csv('../data/data-stations.zip')

## On crée maintenant le dataframe avec les données d'historique
df_bikes = pd.read_csv('../data/data-bikes.zip')

In [ ]:
## On vérifie le type de notre variable
type(df_stations)

In [ ]:
## On affiche la liste des colonnes
df_stations.columns

In [ ]:
## On affiche les premières lignes
df_stations.head()

* Combien y a-t-il de stations velo'v ?

In [ ]:
## On affiche la taille du dataframe
## La méthode shape retourne les dimensions (lignes / colonnes)
print(df_stations.shape)

## La fonction len() retourne le nombre de ligne
print(len(df_stations))

In [4]:
## On affiche les premières lignes
df_bikes.head()

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0


### 2.2. Premier apercu des données d'historique

In [5]:
## On affiche les information sur les données
df_bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7800252 entries, 0 to 7800251
Data columns (total 11 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   id_velov        object
 1   year            int64 
 2   month           int64 
 3   day             int64 
 4   hour            int64 
 5   minute          int64 
 6   time            object
 7   bikes           int64 
 8   bike_stands     int64 
 9   departure30min  int64 
 10  arrival30min    int64 
dtypes: int64(9), object(2)
memory usage: 654.6+ MB


In [6]:
# Réduction de la taille en mémoire

## on transforme le type des colonnes en entier ou float lorsque cela est nécessaire
df_bikes['bikes'] = df_bikes.bikes.apply(lambda x: int(float(x)))
df_bikes.bike_stands = df_bikes.bike_stands.apply(lambda x: np.int32(float(x)))
df_bikes['year'] = df_bikes['year'].astype('int16')
df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']] = df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']].astype('int8')


In [7]:
## On affiche les information sur les données
df_bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7800252 entries, 0 to 7800251
Data columns (total 11 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   id_velov        object
 1   year            int16 
 2   month           int8  
 3   day             int8  
 4   hour            int8  
 5   minute          int8  
 6   time            object
 7   bikes           int8  
 8   bike_stands     int8  
 9   departure30min  int8  
 10  arrival30min    int8  
dtypes: int16(1), int8(8), object(2)
memory usage: 193.4+ MB


In [8]:
## Description des données
df_bikes.describe()

,year,month,day,hour,minute,bikes,bike_stands,departure30min,arrival30min
count,7.800252e+06,7.800252e+06,7.800252e+06,7.800252e+06,7.800252e+06,7.800252e+06,7.800252e+06,7.800252e+06,7.800252e+06
mean,2.021004e+03,5.919969e+00,1.583957e+01,1.133367e+01,1.475726e+01,9.086573e+00,1.132540e+01,5.970303e-01,5.999383e-01
std,6.531360e-02,3.412203e+00,8.716837e+00,7.013109e+00,1.499804e+01,7.668874e+00,8.209105e+00,1.284539e+00,1.286469e+00
min,2.021000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.021000e+03,3.000000e+00,9.000000e+00,5.000000e+00,0.000000e+00,3.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00
50%,2.021000e+03,5.000000e+00,1.600000e+01,1.100000e+01,0.000000e+00,8.000000e+00,1.100000e+01,0.000000e+00,0.000000e+00
75%,2.021000e+03,9.000000e+00,2.300000e+01,1.700000e+01,3.000000e+01,1.400000e+01,1.600000e+01,1.000000e+00,1.000000e+00
max,2.022000e+03,1.200000e+01,3.100000e+01,2.300000e+01,3.000000e+01,5.500000e+01,5.500000e+01,4.800000e+01,5.400000e+01


In [9]:
## On affiche 5 lignes sélectionnées de manière aléatoire
df_bikes.sample(5)

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
1656920,velov-10118,2021,11,6,11,0,2021-11-06 11:00:00+00:00,20,2,1,0
5013033,velov-6004,2021,4,11,4,0,2021-04-11 04:00:00+00:00,8,11,0,0
4722384,velov-5036,2021,8,10,1,30,2021-08-10 01:30:00+00:00,12,3,0,0
2616984,velov-2024,2021,3,18,6,30,2021-03-18 06:30:00+00:00,21,4,0,2
3034996,velov-3012,2021,4,11,20,30,2021-04-11 20:30:00+00:00,16,9,0,0


### 2.3. Manipulation d'un dataframe

In [10]:
## Accès à une colonne
df_bikes['time']

0          2021-01-01 00:00:00+00:00
1          2021-01-01 00:30:00+00:00
2          2021-01-01 01:00:00+00:00
3          2021-01-01 01:30:00+00:00
4          2021-01-01 02:00:00+00:00
                     ...            
7800247    2022-01-01 23:00:00+00:00
7800248    2022-01-01 23:30:00+00:00
7800249    2022-01-01 23:30:00+00:00
7800250    2022-01-02 00:00:00+00:00
7800251    2022-01-02 00:00:00+00:00
Name: time, Length: 7800252, dtype: object

In [11]:
## Accès à une colonne (autre manière en utilisant le .)
df_bikes.time

0          2021-01-01 00:00:00+00:00
1          2021-01-01 00:30:00+00:00
2          2021-01-01 01:00:00+00:00
3          2021-01-01 01:30:00+00:00
4          2021-01-01 02:00:00+00:00
                     ...            
7800247    2022-01-01 23:00:00+00:00
7800248    2022-01-01 23:30:00+00:00
7800249    2022-01-01 23:30:00+00:00
7800250    2022-01-02 00:00:00+00:00
7800251    2022-01-02 00:00:00+00:00
Name: time, Length: 7800252, dtype: object

In [16]:
## Accès à un ensemble de colonnes
mes_colonnes = ['time','bikes','year']
df_bikes[mes_colonnes]

,time,bikes,year
0,2021-01-01 00:00:00+00:00,8,2021
1,2021-01-01 00:30:00+00:00,8,2021
2,2021-01-01 01:00:00+00:00,7,2021
3,2021-01-01 01:30:00+00:00,7,2021
4,2021-01-01 02:00:00+00:00,7,2021
...,...,...,...
7800247,2022-01-01 23:00:00+00:00,3,2022
7800248,2022-01-01 23:30:00+00:00,3,2022
7800249,2022-01-01 23:30:00+00:00,3,2022
7800250,2022-01-02 00:00:00+00:00,4,2022


In [ ]:
## Récupérer les valeurs d'un ensemble de colonnes
df_bikes[['time', 'bikes']].values


array([['2021-01-01 00:00:00+00:00', 8],
       ['2021-01-01 00:30:00+00:00', 8],
       ['2021-01-01 01:00:00+00:00', 7],
       ...,
       ['2022-01-01 23:30:00+00:00', 3],
       ['2022-01-02 00:00:00+00:00', 4],
       ['2022-01-02 00:00:00+00:00', 4]], shape=(7800252, 2), dtype=object)

Une colonne (ou variable) est un vecteur de données (Series dans la terminologie de la librarie Pandas).

In [19]:
## Affichage des premières valeurs d'une seule colonne
df_bikes['time'].head()

0    2021-01-01 00:00:00+00:00
1    2021-01-01 00:30:00+00:00
2    2021-01-01 01:00:00+00:00
3    2021-01-01 01:30:00+00:00
4    2021-01-01 02:00:00+00:00
Name: time, dtype: object

In [20]:
## Affichage des dernières valeurs de la colonne
df_bikes['time'].tail()

7800247    2022-01-01 23:00:00+00:00
7800248    2022-01-01 23:30:00+00:00
7800249    2022-01-01 23:30:00+00:00
7800250    2022-01-02 00:00:00+00:00
7800251    2022-01-02 00:00:00+00:00
Name: time, dtype: object

In [21]:
## On trie les valeurs d'une colonne de manière croissante
df_bikes['time'].sort_values()

0          2021-01-01 00:00:00+00:00
1363382    2021-01-01 00:00:00+00:00
2467362    2021-01-01 00:00:00+00:00
2703137    2021-01-01 00:00:00+00:00
3349345    2021-01-01 00:00:00+00:00
                     ...            
2398034    2022-01-02 00:00:00+00:00
2398035    2022-01-02 00:00:00+00:00
7503119    2022-01-02 00:00:00+00:00
2328699    2022-01-02 00:00:00+00:00
7800251    2022-01-02 00:00:00+00:00
Name: time, Length: 7800252, dtype: object

In [22]:
## Le tri peut également être généralisé aux DataFrame
## Tri du jeu de données selon l'id de la station et la date
df_bikes.sort_values(by=['id_velov', 'time'])

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7800247,velov-9052,2022,1,1,23,0,2022-01-01 23:00:00+00:00,3,9,0,0
7800248,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800249,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800250,velov-9052,2022,1,2,0,0,2022-01-02 00:00:00+00:00,4,8,0,1


In [24]:
## Alternative permettant de remettre l'index des lignes à zéro
df_bikes = df_bikes.sort_values(by=['id_velov', 'time']).reset_index(drop=True)
df_bikes



,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7800247,velov-9052,2022,1,1,23,0,2022-01-01 23:00:00+00:00,3,9,0,0
7800248,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800249,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800250,velov-9052,2022,1,2,0,0,2022-01-02 00:00:00+00:00,4,8,0,1


In [34]:
## Comptage des valeurs
df_bikes['id_velov'].value_counts()

id_velov
velov-1013     23392
velov-1005     23389
velov-1006     23388
velov-1003     23388
velov-1016     23388
               ...  
velov-10049    17991
velov-7012     15484
velov-9014      9342
velov-2023      5366
velov-3001       336
Name: count, Length: 343, dtype: int64

In [35]:
## une colonne étant un vecteur il est possible d'utiliser des indices pour accèder aux éléments
## Affichage de la première valeur de la colonne time
df_bikes['time'][0]

'2021-01-01 00:00:00+00:00'

In [36]:
## Affichage des 3 premières valeurs de la colonne time
df_bikes['time'][0:3]

0    2021-01-01 00:00:00+00:00
1    2021-01-01 00:30:00+00:00
2    2021-01-01 01:00:00+00:00
Name: time, dtype: object

#### 2.3.1 Itérations sur les colonnes (variables)

Les itérations sur les variables peuvent se faire via une boucle, ou via l'utilisation de fonctions callback appelée à l'aide d'une fonction `.apply()`.


In [ ]:
## Boucler sur l'ensemble des colonne pour afficher leur nom et leur type
for col in df_bikes.columns:
    print(col, ": ", df_bikes[col].dtype) 


id_velov
year
month
day
hour
minute
time
bikes
bike_stands
departure30min
arrival30min


#### 2.3.2 Itérations sur les lignes (\*\*déconseillé dans le cas des grands dataframe**)


Il est possible de parcourir les lignes d'un dataframe, mais attention, l'itération sur un dataframe est lent. Mieux vaut utiliser des opérations vectorielles ! Si on ne peut pas, on préfére utiliser une fonction callback appelée à l'aide d'une fonction `.apply()`.

Remarque : on ne peut pas modifier un dataframe sur lequel on boucle.



In [42]:
## Pour l'exemple, on itère sur le dataframe des stations (car celui de l'historique est trop grand)
for index, row in df_stations.iterrows():
    print('ID :', row.id_velov, '\t lat :', row.latitude,'\t lng :', row.longitude)




ID : velov-10056 	 lat : 45.779112 	 lng : 4.871952
ID : velov-9013 	 lat : 45.787384 	 lng : 4.814374
ID : velov-5044 	 lat : 45.759797 	 lng : 4.796627
ID : velov-9014 	 lat : 45.783341 	 lng : 4.811433
ID : velov-10058 	 lat : 45.767126 	 lng : 4.89215
ID : velov-9022 	 lat : 45.778554 	 lng : 4.80704
ID : velov-10060 	 lat : 45.758788 	 lng : 4.878976
ID : velov-2009 	 lat : 45.739608 	 lng : 4.815074
ID : velov-10034 	 lat : 45.761788 	 lng : 4.886157
ID : velov-2036 	 lat : 45.75947 	 lng : 4.830145
ID : velov-6006 	 lat : 45.77332 	 lng : 4.85183
ID : velov-1016 	 lat : 45.765949 	 lng : 4.831107
ID : velov-6001 	 lat : 45.766855 	 lng : 4.858974
ID : velov-9049 	 lat : 45.770134 	 lng : 4.805158
ID : velov-5026 	 lat : 45.763376 	 lng : 4.82904
ID : velov-6008 	 lat : 45.768412 	 lng : 4.859071
ID : velov-9040 	 lat : 45.771006 	 lng : 4.807613
ID : velov-10005 	 lat : 45.779702 	 lng : 4.859991
ID : velov-2038 	 lat : 45.75308 	 lng : 4.829649
ID : velov-1020 	 lat : 45.766806

#### 2.3.3 Accès indicé aux données d'un DataFrame

On peut accéder aux valeurs du DataFrame via des indices ou plages d'indices. 
La structure se comporte alors comme une matrice. La cellule en haut à gauche est de coordonnées (0,0).
Il y a différentes manières de le faire, l'utilisation de `.iloc[,]` constitue une des solutions les plus simples.
Rappel : la méthode `shape()` permet d'obtenir les dimensions (lignes et colonnes) du DataFrame.


In [48]:
## Accès à la valeur située en (0,0) (première ligne, première colonne)
df_bikes.iloc[0:2,0]

0    velov-10001
1    velov-10001
Name: id_velov, dtype: object

In [49]:
## Valeur située en dernière ligne, première colonne
## Utilisation de l'indiçage négatif
df_bikes.iloc[-1,0]

'velov-9052'

In [ ]:
## Alternative avec shape, valeur située en dernière ligne, première colonne
## shape[0] renvoie le nombre de lignes (première dimension)
## il faut réduire de -1 parce le premier indice est égal à 0 sinon on déborde
df_bikes.iloc[df_bikes.shape[0]-1,0]
df_bikes.shape[0]

7800252

In [57]:
## Affichage des 5 premières valeurs de toutes les colonnes
## lignes => 0:5 (0 à 5 [non inclus])
## colonnes = : (toutes les colonnes)
df_bikes.iloc[0:5,:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0


In [58]:
## Avec l'indiçage négatif, on peut facilement accéder aux 5 dernières lignes
df_bikes.iloc[-5:,:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
7800247,velov-9052,2022,1,1,23,0,2022-01-01 23:00:00+00:00,3,9,0,0
7800248,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800249,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800250,velov-9052,2022,1,2,0,0,2022-01-02 00:00:00+00:00,4,8,0,1
7800251,velov-9052,2022,1,2,0,0,2022-01-02 00:00:00+00:00,4,8,0,1


In [60]:
## 5 premières lignes et colonnes 0, 6, 7 et 8
## on a une liste d'indices en colonne
df_bikes.iloc[0:5,[0,7,6,8]]

,id_velov,bikes,time,bike_stands
0,velov-10001,8,2021-01-01 00:00:00+00:00,22
1,velov-10001,8,2021-01-01 00:30:00+00:00,22
2,velov-10001,7,2021-01-01 01:00:00+00:00,23
3,velov-10001,7,2021-01-01 01:30:00+00:00,23
4,velov-10001,7,2021-01-01 02:00:00+00:00,23


#### 2.3.4 Filtrage avec des conditions - Les requêtes

Nous pouvons isoler les sous-ensembles d'observations répondant à des critères définis sur les champs. Nous utiliserons préférentiellement la méthode `.loc[,]` dans ce cadre.

In [61]:
## Liste des données d'historique pour la station 'velov-10001'
df_bikes.loc[df_bikes['id_velov']=="velov-10001",:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0
...,...,...,...,...,...,...,...,...,...,...,...
22848,velov-10001,2022,1,1,23,0,2022-01-01 23:00:00+00:00,7,21,0,0
22849,velov-10001,2022,1,1,23,30,2022-01-01 23:30:00+00:00,7,21,0,0
22850,velov-10001,2022,1,1,23,30,2022-01-01 23:30:00+00:00,7,21,0,0
22851,velov-10001,2022,1,2,0,0,2022-01-02 00:00:00+00:00,7,21,0,0


In [64]:
## Pour un ensemble de valeurs de la même variable, on utilise la méthode isin()
mes_stations = ['velov-10001','velov-10002']

df_bikes.loc[df_bikes['id_velov'].isin(mes_stations),:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0
...,...,...,...,...,...,...,...,...,...,...,...
41215,velov-10002,2022,1,1,23,0,2022-01-01 23:00:00+00:00,3,52,0,0
41216,velov-10002,2022,1,1,23,30,2022-01-01 23:30:00+00:00,5,50,0,2
41217,velov-10002,2022,1,1,23,30,2022-01-01 23:30:00+00:00,5,50,0,1
41218,velov-10002,2022,1,2,0,0,2022-01-02 00:00:00+00:00,5,50,0,0


Des opérateurs logiques permettent de combiner les conditions. 
On utilise respectivement : & pour ET, | pour OU, et ~ pour la négation.

In [65]:
## Liste des données pour la station 'velov-10001' et hour = 8
df_bikes.loc[(df_bikes['id_velov']=="velov-10001") & (df_bikes['hour'] == 8),:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
16,velov-10001,2021,1,1,8,0,2021-01-01 08:00:00+00:00,7,23,0,0
17,velov-10001,2021,1,1,8,30,2021-01-01 08:30:00+00:00,7,23,0,0
65,velov-10001,2021,1,2,8,0,2021-01-02 08:00:00+00:00,5,25,0,0
66,velov-10001,2021,1,2,8,30,2021-01-02 08:30:00+00:00,5,25,0,0
114,velov-10001,2021,1,3,8,0,2021-01-03 08:00:00+00:00,8,22,0,0
...,...,...,...,...,...,...,...,...,...,...,...
22694,velov-10001,2021,12,31,8,30,2021-12-31 08:30:00+00:00,6,24,0,0
22787,velov-10001,2022,1,1,8,0,2022-01-01 08:00:00+00:00,7,22,0,0
22788,velov-10001,2022,1,1,8,0,2022-01-01 08:00:00+00:00,7,22,0,0
22789,velov-10001,2022,1,1,8,30,2022-01-01 08:30:00+00:00,7,22,0,0


In [66]:
## Liste des données datant d'après juillet
df_bikes.loc[(df_bikes['month'] > 7),:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
14839,velov-10001,2021,8,1,0,0,2021-08-01 00:00:00+00:00,15,15,0,1
14840,velov-10001,2021,8,1,0,0,2021-08-01 00:00:00+00:00,15,15,0,1
14841,velov-10001,2021,8,1,0,30,2021-08-01 00:30:00+00:00,15,15,0,0
14842,velov-10001,2021,8,1,1,0,2021-08-01 01:00:00+00:00,15,15,0,0
14843,velov-10001,2021,8,1,1,30,2021-08-01 01:30:00+00:00,15,15,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7800149,velov-9052,2021,12,31,22,30,2021-12-31 22:30:00+00:00,6,6,0,0
7800150,velov-9052,2021,12,31,23,0,2021-12-31 23:00:00+00:00,6,6,0,0
7800151,velov-9052,2021,12,31,23,0,2021-12-31 23:00:00+00:00,6,6,0,0
7800152,velov-9052,2021,12,31,23,30,2021-12-31 23:30:00+00:00,6,6,0,0


In [68]:
#on peut n'afficher qu'une partie des colonnes
#on définit la projection dans une liste
colonnes = ['id_velov','time','bikes','bike_stands']
#que l'on utilise en paramètre dans .loc[]
#pour la même restruction que précédemment
df_bikes.loc[(df_bikes['month'] > 7),colonnes]

,id_velov,time,bikes,bike_stands
14839,velov-10001,2021-08-01 00:00:00+00:00,15,15
14840,velov-10001,2021-08-01 00:00:00+00:00,15,15
14841,velov-10001,2021-08-01 00:30:00+00:00,15,15
14842,velov-10001,2021-08-01 01:00:00+00:00,15,15
14843,velov-10001,2021-08-01 01:30:00+00:00,15,15
...,...,...,...,...
7800149,velov-9052,2021-12-31 22:30:00+00:00,6,6
7800150,velov-9052,2021-12-31 23:00:00+00:00,6,6
7800151,velov-9052,2021-12-31 23:00:00+00:00,6,6
7800152,velov-9052,2021-12-31 23:30:00+00:00,6,6


* Combien de stations vélo'v on eu plus de 5 départs le 20 septembre 2021 à 12h00 ? 

In [73]:
filtered = df_bikes.loc[(df_bikes['year']==2021) & (df_bikes['month']==9) & (df_bikes['day']==20) & (df_bikes['hour']==12) & (df_bikes['minute']==00) & (df_bikes['departure30min']>5),:]
filtered.shape

(13, 11)

#### 2.3.5 Regroupement des variables

L'utilisation de `groupby()` permet d'accéder aux sous-DataFrame associés à chaque item de la variable de regroupement. Il est dès lors possible d'appliquer explicitement d'autres traitements sur ces sous-ensembles de données.

In [74]:
#regroupement des données selon le l'id de la station
g = df_bikes.groupby(['id_velov'])

g.size()

id_velov
velov-10001    22853
velov-10002    18367
velov-10004    22868
velov-10005    22891
velov-10006    22858
               ...  
velov-9044     22866
velov-9049     22866
velov-9050     22854
velov-9051     22853
velov-9052     22855
Length: 343, dtype: int64

In [76]:
#calculer la dimension du sous-DataFrame associé à la station 'velov_10001'
g.get_group('velov-10001').shape

C:\Users\BastienCAHIER\AppData\Local\Temp\ipykernel_10740\301033285.py:2: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  g.get_group('velov-10001').shape


(22853, 11)

### 2.4. Visualisation des localisations des stations

Maintenant que vous avez chargé les données en mémoire et vue comment manipuler un `DataFrame`, vous allez produire votre première carte.



#### 2.4.1 Utilisation des librairies GeoPandas et Plotly 

La librairie [GeoPandas](https://geopandas.org/) est conçu pour faciliter la manipulation de données spatiales. La particularité de GeoPandas est qu'elle permet de manipuler les données spatiales comme s'il s'agissait de données traditionnelles. 

Par rapport à un `DataFrame` standard, un `GeoDataFrame`, comporte une colonne supplémentaire: `geometry`. Comme dans un SGBD spatial, cette colonne permet de stocker les contours (la géométrie) d'un objet géographique. Un objet `GeoDataFrame` hérite des propriétés d'un `DataFrame` mais propose des méthodes adaptées au traitement des données spatiales.

Ainsi en plus des manipulations déjà possible avec pandas, on pourra manipulation la dimension spatiale : 
- calculer des distances et des surfaces,
- agréger rapidement des zonages (regrouper les départements en région par exemple),
- rechercher une zone à partir des coordonnées d'un point,
- convertir les données dans différents systèmes de projection,
- faire une carte.

Pour le moment on s'intèresse au dernier point afin de produire une carte des stations Velo'v.

![stations velov avec GeoPandas](https://perso.liris.cnrs.fr/lmoncla/GEONUM/fig/geopandas_stations.png)

* Affichez les stations vélo'v sur une carte. Utiliser la librairie [GeoPandas](https://geopandas.org/gallery/create_geopandas_from_pandas.html#sphx-glr-gallery-create-geopandas-from-pandas-py). Vous devez obtenir le résultat ci-dessus.

In [79]:
## On transforme le dataframe des stations en geodataframe (https://geopandas.org/gallery/create_geopandas_from_pandas.html#sphx-glr-gallery-create-geopandas-from-pandas-py)
gdf_stations = gpd.GeoDataFrame(
    df_stations, 
    geometry=gpd.points_from_xy(df_stations.longitude, df_stations.latitude))



In [80]:
## On affiche les premières lignes du GeoDataFrame pour vérifier l'existance de la colonne géométrie
gdf_stations.head()

,id_velov,latitude,longitude,geometry
0,velov-10056,45.779112,4.871952,POINT (4.87195 45.77911)
1,velov-9013,45.787384,4.814374,POINT (4.81437 45.78738)
2,velov-5044,45.759797,4.796627,POINT (4.79663 45.7598)
3,velov-9014,45.783341,4.811433,POINT (4.81143 45.78334)
4,velov-10058,45.767126,4.892150,POINT (4.89215 45.76713)


In [82]:
## On affiche directement les données du geodataframe sur une carte 
## avec la méthode scatter_mapbox() de la librairie plotly.express:
fig = px.scatter_mapbox(gdf_stations,
                        lat=gdf_stations.geometry.y,
                        lon=gdf_stations.geometry.x,
                        hover_name="id_velov",
                        zoom=12, mapbox_style="carto-positron")

## On supprime les marges autour de la carte
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

## On affiche la carte
fig.show()

## 3. Préparation des données

### 3.1. Ajout d'information

Avant de pouvoir analyser les données d'historique, on souhaite ajouter quelques informations. Par exemple, le jeu de données initial ne fournit pas directement les trajets (départs / arrivées) des utilisateurs mais seulement le nombre de vélos ou de places disponibles à un instant t (par tranche de 5 minutes). Pour faire une analyse de la fréquentation ou des zones de départs et d'arrivées en fonction du moment de la journée ou de la semaine j'ai calculé les départs et arrivées par tranches de 30min.

A partir des tranches de 30 min nous pouvons par exemple inférer le nombre quotidien.


In [ ]:
## On commence par faire une copie de notre DataFrame, pour pouvoir revenir aux données initiales si besoin
df_sampled = df_bikes.copy()

#### 3.1.1 Calcul du nombre d'arrivées et de départs quotidiens


On peut faire des calculs directement en groupant les lignes grâce à la méthode `groupby()`.

Quelles colonnes faut-il regrouper pour pouvoir calculer les départs et arrivées quotidiens ?

In [84]:
df_sampled.head()

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0


In [85]:
## La méthode 'transform' permet d'appliquer un calcul au dataframe d'origine (non groupé). 
## Dans notre cas, on souhaite effectuer une somme sur les colonnes departure30min et arrival30min.

## Compléter la liste des colonnes
df_sampled["daily_departure"] = df_sampled.groupby(['id_velov','year','month','day'])['departure30min'].transform('sum')
df_sampled["daily_arrival"] = df_sampled.groupby(['id_velov','year','month','day'])['arrival30min'].transform('sum')


In [ ]:
## On affiche 15 lignes aléatoirement pour visualiser le résultat
df_sampled.sample(15)


,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min,daily_departure,daily_arrival
3132814,velov-3018,2021,7,11,6,30,2021-07-11 06:30:00+00:00,26,4,0,1,60,46
5724493,velov-7007,2021,4,27,13,30,2021-04-27 13:30:00+00:00,0,20,1,0,45,43
3716331,velov-3089,2021,11,26,5,30,2021-11-26 05:30:00+00:00,4,20,0,0,36,39
158102,velov-1001,2021,2,12,20,30,2021-02-12 20:30:00+00:00,13,2,0,0,58,51
3151812,velov-3021,2021,5,6,14,30,2021-05-06 14:30:00+00:00,12,8,0,3,36,37
5242278,velov-6022,2021,4,11,15,30,2021-04-11 15:30:00+00:00,9,10,0,0,40,25
7218177,velov-8058,2021,12,9,12,0,2021-12-09 12:00:00+00:00,13,3,1,4,35,36
3897086,velov-3102,2021,9,15,1,30,2021-09-15 01:30:00+00:00,6,10,0,0,19,12
3200985,velov-3029,2021,6,21,2,0,2021-06-21 02:00:00+00:00,23,15,0,0,68,85
3214422,velov-3031,2021,2,16,9,0,2021-02-16 09:00:00+00:00,12,1,0,0,35,35


#### 3.1.2 Distinction semaine / weekend

Afin d'analyser les données on souhaite pouvoir distinguer les jours de la semaine des jours de weekend, pour cela nous devons préparer les données afin d'identifier les jours de weekend.

1. On défini une fonction qui retourne vrai lorsque la date est un jour de la semaine et faux lorsque c'est le weekend
2. On applique cette fonction sur chaque ligne de notre dataframe

In [106]:
## La fonction weekDay, prend 3 paramètres : l'année, le mois et le jour
def weekDay(year, month, day):
    ## Cette méthode retourne vrai (True) si la date correspond à un jour de la semaine, faux (False) sinon
    ## On utilise la fonction datetime() et la méthode weekday()
    ## https://docs.python.org/fr/3/library/datetime.html#datetime.datetime
    
    as_date = datetime(year,month,day)
    if as_date.weekday() < 5:
        return True
    else:
        return False

## On vectorise la fonction afin de l'appliquer de manière efficace (en terme de temps de calcul) sur le dataframe
isWeekDay = np.vectorize(weekDay)

In [105]:
## On ajoute une nouvelle colonne à partir du résultat de la fonction appliquée sur l'ensemble des lignes du dataframe
df_sampled['IsWeekday'] = isWeekDay(df_sampled['year'],df_sampled['month'],df_sampled['day'])


In [102]:
## on affiche un échantillon du dataframe
df_sampled.sample(10)

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min,daily_departure,daily_arrival,IsWeekday
5590607,velov-7001,2021,6,20,22,0,2021-06-20 22:00:00+00:00,11,9,4,0,45,50,False
4256797,velov-4023,2021,4,23,15,30,2021-04-23 15:30:00+00:00,7,15,0,1,32,33,True
6631596,velov-8008,2021,4,8,14,0,2021-04-08 14:00:00+00:00,13,3,1,0,20,11,True
5656361,velov-7004,2021,5,6,22,0,2021-05-06 22:00:00+00:00,30,0,0,1,51,47,True
5545506,velov-6044,2021,7,5,11,0,2021-07-05 11:00:00+00:00,5,10,0,5,47,44,True
206462,velov-10012,2021,3,4,6,30,2021-03-04 06:30:00+00:00,5,11,0,0,32,53,True
2553889,velov-2017,2021,9,8,16,0,2021-09-08 16:00:00+00:00,3,22,4,2,104,103,True
6602922,velov-8007,2021,1,29,19,30,2021-01-29 19:30:00+00:00,2,12,1,0,41,37,True
2584484,velov-2022,2021,2,2,2,30,2021-02-02 02:30:00+00:00,9,21,0,0,20,35,True
5870455,velov-7014,2021,2,11,19,30,2021-02-11 19:30:00+00:00,11,19,0,2,60,50,True


* Créer une nouvelle colonne `day_of_week` qui contient le jour de la semaine (0 pour lundi, 1 pour mardi, etc.)

In [109]:
df_sampled['day_of_week'] = df_sampled.apply(lambda row: datetime(row['year'], row['month'], row['day']).weekday(), axis=1)

In [110]:
## on affiche un échantillon du dataframe
df_sampled.sample(10)

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min,daily_departure,daily_arrival,IsWeekday,day_of_week
4522327,velov-5007,2021,12,15,13,30,2021-12-15 13:30:00+00:00,3,13,1,0,27,26,True,2
5314750,velov-6028,2021,6,2,16,30,2021-06-02 16:30:00+00:00,0,16,1,1,62,57,True,2
171526,velov-1001,2021,8,14,11,30,2021-08-14 11:30:00+00:00,12,4,1,0,50,51,False,5
3305187,velov-3043,2021,2,2,17,0,2021-02-02 17:00:00+00:00,5,11,1,6,44,48,True,1
2789346,velov-2037,2021,9,3,8,0,2021-09-03 08:00:00+00:00,17,22,3,3,94,83,True,4
36797,velov-10002,2021,10,8,12,0,2021-10-08 12:00:00+00:00,46,9,1,42,174,159,True,4
2482643,velov-2014,2021,8,5,0,0,2021-08-05 00:00:00+00:00,9,17,1,0,53,55,True,3
2722215,velov-2030,2021,10,16,22,30,2021-10-16 22:30:00+00:00,24,7,0,4,102,89,False,5
3754420,velov-3091,2021,6,23,20,30,2021-06-23 20:30:00+00:00,0,20,0,0,39,45,True,2
3051998,velov-3013,2021,2,11,4,30,2021-02-11 04:30:00+00:00,3,18,0,0,24,24,True,3


## 4. Sauvegarde du jeu de données préparé

Afin de pouvoir réutiliser le jeu de données sans refaire tous les traitements on l'enregistre dans un fichier CSV.

Utiliser la méthode [to_csv()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) de la librairie Pandas pour enregistrer le dataframe modifié dans un fichier.

In [111]:
## On enregistre le dataframe modifié

compression_opts = dict(method='zip', archive_name='data-bikes-2.csv')  
df_sampled.to_csv('../data/data-bikes-2.zip', index=False, compression=compression_opts)